In [2]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langgraph.graph import StateGraph, END, START
from dotenv import load_dotenv
from typing import TypedDict

In [3]:
load_dotenv()

True

In [4]:
llm=HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1", 
    
)
model= ChatHuggingFace(llm=llm)

In [5]:
class Blogstate(TypedDict):

    title : str
    outline :  str
    content : str

In [6]:
graph=StateGraph(Blogstate)


In [7]:
def outline_node(state: Blogstate)->Blogstate:
    title=state['title']
    prompt=f"Generate a outline for the given topic-{title}"
    state['outline']=model.invoke(prompt).content
    return state


In [8]:
def blog_func(state: Blogstate)->Blogstate:
    prompt=f"Generate a detailed blog by using this topic-{state['title']} & given outline\n {state['outline']}"
    output=model.invoke(prompt).content
    state['content']=output
    return state

In [9]:
graph.add_node("outline node",outline_node)
graph.add_node("Blog node",blog_func)

graph.add_edge(START,"outline node")
graph.add_edge("outline node","Blog node")
graph.add_edge("Blog node",END)

In [11]:
workflow=graph.compile()

In [13]:
output=workflow.invoke({"title":"Rohit Sharma"})
print(output)

{'title': 'Rohit Sharma', 'outline': 'Of course! Here is a detailed outline for the topic "Rohit Sharma," structured to cover his life and career comprehensively. This outline can be adapted for a biography, a research paper, a presentation, or a detailed article.\n\n### **Title: Rohit Sharma: The Hitman\'s Journey from Prodigy to Legend**\n\n---\n\n### **I. Introduction**\n    A. Opening Hook: Start with a defining moment (e.g., his third double-century, 2019 WC century vs. Pakistan, or being appointed captain).\n    B. Brief Overview: Introduce Rohit Sharma as one of the most destructive and successful batters in modern cricket.\n    C. Thesis Statement: Rohit Sharma\'s career is a remarkable story of prodigious talent, early setbacks, incredible redemption in limited-overs cricket, and ultimate evolution into a successful leader of the Indian cricket team.\n\n### **II. Early Life and Background**\n    A. Birth and Family: Born April 30, 1987, in Bansod, Nagpur, Maharashtra.\n    B. 

In [15]:
output['content']

'Of course. Here is a detailed blog post crafted from the provided outline on Rohit Sharma.\n\n***\n\n## **Rohit Sharma: The Hitman\'s Journey from Prodigy to Legend**\n\n### **I. Introduction**\n\nThe roar of the MCG crowd was deafening, a sea of blue waving frantically as the ball sailed effortlessly over the long-on boundary. It wasn’t just a six; it was a statement. In a high-pressure World Cup clash against Pakistan in 2019, with the weight of a billion expectations on his shoulders, Rohit Sharma compiled a masterful, unhurried 140. It was an innings that encapsulated his entire career: sublime talent, calm composure, and the ability to rise to the grandest occasions.\n\nRohit Gurunath Sharma is not just a cricketer; he is a phenomenon. He is the architect of carnage, the maker of records, and the gentle giant who redefined the art of limited-overs batting. His career is a cinematic tale of a prodigy burdened by the "talent" tag, who faced years of frustration before a strategic m

In [16]:
## generate the evaluation score